In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import mean_absolute_error
from copy import deepcopy
sns.set()

In [4]:
X_train = pd.read_csv('../input/project/lt_train.csv/lt_train.csv')
X_test = pd.read_csv('../input/project/lt_test.csv')

In [5]:
df = X_train.loc[X_train['demand'].isnull() == False]

In [6]:
#посмотрим на период данных для обучения
print(df['period_dt'].min())
print(df['period_dt'].max())

2016-12-19
2018-12-24


In [7]:
#посмотрим на период данных для предсказания
print(X_test['period_dt'].min())
print(X_test['period_dt'].max())

2019-01-07
2019-12-30


In [8]:
X_train['Date'] = pd.to_datetime(X_train['period_dt'], dayfirst = True)
X_train.set_index(['Date'], inplace=True)
X_train.drop('period_dt', axis=1, inplace=True)
X_train.head()

,location_id,product_id,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,id
Date,,,,,,,,,,
2019-12-16,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,1
2019-12-30,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,1.0,3
2019-12-09,764.0,23285.0,NaN,0.0,0.0,2199.0,1099.5,0.0,0.0,4
2019-12-02,453.0,23285.0,NaN,1.0,0.0,2199.0,1319.4,0.0,1.0,5
2019-12-02,764.0,23285.0,NaN,1.0,0.0,2199.0,1099.5,0.0,0.0,6


In [26]:
#создадим признаки, обозначающие день, месяц и год для наблюдения
data = X_train.reset_index().copy(deep = True)

data["monthday"] = data.Date.dt.day
data["month"] = data.Date.dt.month
data["year"] = data.Date.dt.year

#прологарифмируем целевую переменную (без логарифма работает хуже) 
data = data.assign(ln_demand = np.log(data['demand'] + 1)) 

In [27]:
data.drop('Date', axis=1, inplace=True)

In [28]:
#заполним пропуски в спросе 
data['demand'] = data['demand'].fillna(-10)

In [29]:
#создадим признак Promo, равный 1, если хотя бы один из вариантов промо присутствовал
data['PROMO1_FLAG'] = data['PROMO1_FLAG'].fillna(0)
data['PROMO2_FLAG'] = data['PROMO2_FLAG'].fillna(0)
data['Promo'] = data['PROMO1_FLAG'] + data['PROMO2_FLAG']

def scaler(x):
    if x >=1:
        x = 1
    else:
        x = 0 
    return x 
#в PROMO1_FLAG есть значения больше 1, поэтому приравняем полученные суммы больше 1 к 1 
data['Promo'] = data['Promo'].apply(lambda x: scaler(x))

In [30]:
data = data.drop(['PROMO1_FLAG', 'PROMO2_FLAG'], axis=1)

In [31]:
#создадим переменную, обозначающую фактическую цену товара 
data['actual_price'] = data['PRICE_REGULAR']
data['actual_price'].loc[data['Promo'] == 1] = data['PRICE_AFTER_DISC']
data = data.drop(['PRICE_REGULAR', 'PRICE_AFTER_DISC'], axis=1)
data.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,location_id,product_id,demand,NUM_CONSULTANT,AUTORIZATION_FLAG,id,monthday,month,year,ln_demand,Promo,actual_price
0,764.0,23285.0,-10.0,0.0,1.0,1,16,12,2019,NaN,0,2199.0
1,764.0,23285.0,-10.0,0.0,1.0,3,30,12,2019,NaN,0,2199.0
2,764.0,23285.0,-10.0,0.0,0.0,4,9,12,2019,NaN,0,2199.0
3,453.0,23285.0,-10.0,0.0,1.0,5,2,12,2019,NaN,1,1319.4
4,764.0,23285.0,-10.0,0.0,0.0,6,2,12,2019,NaN,1,1099.5


In [32]:
data['AUTORIZATION_FLAG'] = data['AUTORIZATION_FLAG'].fillna(1)
data['NUM_CONSULTANT'] = data['NUM_CONSULTANT'].fillna(0)

In [33]:
data.isnull().sum()

location_id          3001944
product_id           3001944
demand                     0
NUM_CONSULTANT             0
AUTORIZATION_FLAG          0
id                         0
monthday                   0
month                      0
year                       0
ln_demand            3155253
Promo                      0
actual_price         1240933
dtype: int64

In [34]:
#уберем наблюдения, по которым нет данных для товаров и магазинов 
data = data.loc[data['product_id'].isnull() == False]
data.isnull().sum()

location_id               0
product_id                0
demand                    0
NUM_CONSULTANT            0
AUTORIZATION_FLAG         0
id                        0
monthday                  0
month                     0
year                      0
ln_demand            153309
Promo                     0
actual_price          13384
dtype: int64

In [35]:
#заполним оставшиеся пропуски нулями
data = data.fillna(0)

In [36]:
le = LabelEncoder()
le.fit(data['location_id'])
data['location_id'] = le.transform(data['location_id'])
le.fit(data['product_id'])
data['product_id'] = le.transform(data['product_id'])

In [38]:
sc = MinMaxScaler()
sc.fit(data.iloc[:, 3:5])
data.iloc[:, 3:5] = sc.transform(data.iloc[:, 3:5])

In [18]:
#добавление лаговых фичей ухудшает результат, поэтому не используется в итоговой версии 

#def lag_features(dataframe, lags):
#    for lag in lags:
#        dataframe['demand_lag_' + str(lag)] = dataframe.groupby(["location_id", "product_id"])['demand'].transform(
#            lambda x: x.shift(lag))
#    return dataframe

#df = lag_features(data, [35, 87, 52])

In [40]:
#data_test = df
#data = df.loc[df['demand'] > -1]

data_test = data
data = data.loc[data['demand'] > -1]

In [42]:
#найдем новые товары 
data_new_id = []

for i in tqdm(range(1, 42)):
    data_0 = data[(data['location_id'] == i)]
    news = data_0['product_id'].tolist()
    for j in range(1, len(news)):
        if j in news:
            data_1 = data_0[(data_0['product_id'] == j)]
            if data_1.shape[0] < 10:
                data_new_id.extend(data_1['id'].tolist())
            else:
                data_1 = data_1.loc[0:10, :]
                data_new_id.extend(data_1['id'].tolist())

100%|██████████| 41/41 [00:38<00:00,  1.07it/s]


In [43]:
#оставим в обучающей выборке только новые товары 
data = data[data['id'].isin(data_new_id)]
test_id_list = X_test['id'].tolist()

In [44]:
data['demand'].describe()
#возьмем порог 0.7 для высокого спроса (медиана)

count    50483.000000
mean         0.710211
std          0.929826
min          0.000000
25%          0.036269
50%          0.700000
75%          1.000000
max         27.000000
Name: demand, dtype: float64

In [45]:
#создадим признак, равный 1, если спрос был высокий, и 0 иначе
data['is_high'] = np.zeros(len(data))
data['is_high'].loc[data['demand'] >= 0.7] = 1
data_high = data.loc[data['is_high'] == 1]
data_low = data.loc[data['is_high'] == 0]
print(len(data_high))
print(len(data_low))

25255
25228


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [46]:
data.drop('demand', axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [47]:
#функция кросс-валидации
def CV(X_train, y_train, number_folds, model, metrics, kwargs={}):

    k = int(np.floor(float(X_train.shape[0]) / number_folds))

    errors = np.zeros(number_folds-1)
 
    for i in range(2, number_folds + 1):
        print('')
        split = float(i-1)/i

        X = X_train[:(k*i)]
        y = y_train[:(k*i)]

        index = int(np.floor(X.shape[0] * split))
       
        X_trainFolds = X[:index]        
        y_trainFolds = y[:index]

        X_testFold = X[(index + 1):]
        y_testFold = y[(index + 1):]

        model.fit(X_trainFolds, y_trainFolds, **kwargs, eval_set=[(X_testFold, y_testFold)])
        errors[i-2] = metrics(model.predict(X_testFold), y_testFold)
    
    return errors.mean()

In [48]:
#будем предсказывать переменную высокого спроса 
y_train_clf = data.is_high
X_train_clf = data.drop(['id','ln_demand','is_high'], axis=1) 

In [52]:
import lightgbm as lgb
import xgboost as xgb 
import catboost
from catboost import CatBoostClassifier

In [50]:
#для предсказания обучим классификатор 
#вариант 1 
lgbm_clf = lgb.LGBMClassifier(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40)    
kwargs={'early_stopping_rounds':10,'verbose':2}

In [53]:
#вариант 2
catboost_clf = CatBoostClassifier(iterations=350,
                         learning_rate=0.1)

In [54]:
#вариант 3 
xgb_clf = xgb.XGBClassifier(n_estimators=350,
                        learning_rate=0.1)

In [55]:
CV(X_train_clf, y_train_clf, 20, lgbm_clf, roc_auc_score, kwargs)
CV(X_train_clf, y_train_clf, 20, catboost_clf, roc_auc_score, kwargs)
CV(X_train_clf, y_train_clf, 20, xgb_clf, roc_auc_score, kwargs)


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's binary_logloss: 0.652244
[4]	valid_0's binary_logloss: 0.627072
[6]	valid_0's binary_logloss: 0.607432
[8]	valid_0's binary_logloss: 0.591973
[10]	valid_0's binary_logloss: 0.581614
[12]	valid_0's binary_logloss: 0.572712
[14]	valid_0's binary_logloss: 0.560873
[16]	valid_0's binary_logloss: 0.55326
[18]	valid_0's binary_logloss: 0.557987
[20]	valid_0's binary_logloss: 0.556993
[22]	valid_0's binary_logloss: 0.561319
[24]	valid_0's binary_logloss: 0.556258
[26]	valid_0's binary_logloss: 0.552012
[28]	valid_0's binary_logloss: 0.553449
[30]	valid_0's binary_logloss: 0.558681
[32]	valid_0's binary_logloss: 0.561482
[34]	valid_0's binary_logloss: 0.559165
[36]	valid_0's binary_logloss: 0.56551

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's binary_logloss: 0.6542

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[8]	valid_0's binary_logloss: 0.557755
[10]	valid_0's binary_logloss: 0.548333
[12]	valid_0's binary_logloss: 0.536489
[14]	valid_0's binary_logloss: 0.523629
[16]	valid_0's binary_logloss: 0.515593
[18]	valid_0's binary_logloss: 0.506254
[20]	valid_0's binary_logloss: 0.49458
[22]	valid_0's binary_logloss: 0.487145
[24]	valid_0's binary_logloss: 0.47827
[26]	valid_0's binary_logloss: 0.475515
[28]	valid_0's binary_logloss: 0.471203
[30]	valid_0's binary_logloss: 0.468465
[32]	valid_0's binary_logloss: 0.46575
[34]	valid_0's binary_logloss: 0.463252
[36]	valid_0's binary_logloss: 0.460682
[38]	valid_0's binary_logloss: 0.458714
[40]	valid_0's binary_logloss: 0.458378
[42]	valid_0's binary_logloss: 0.458659
[44]	valid_0's binary_logloss: 0.455372
[46]	valid_0's binary_logloss: 0.454925
[48]	valid_0's binary_logloss: 0.453769
[50]	valid_0's binary_logloss: 0.452361
[52]	valid_0's binary_logloss: 0.451743
[54]	valid_0's binary_logloss: 0.451339
[56]	valid_0's binary_logloss: 0.449184
[58]

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[46]	valid_0's binary_logloss: 0.498655
[48]	valid_0's binary_logloss: 0.4985
[50]	valid_0's binary_logloss: 0.494548
[52]	valid_0's binary_logloss: 0.496111
[54]	valid_0's binary_logloss: 0.495416
[56]	valid_0's binary_logloss: 0.49418
[58]	valid_0's binary_logloss: 0.492926
[60]	valid_0's binary_logloss: 0.49194
[62]	valid_0's binary_logloss: 0.491649
[64]	valid_0's binary_logloss: 0.497669
[66]	valid_0's binary_logloss: 0.496191
[68]	valid_0's binary_logloss: 0.496227
[70]	valid_0's binary_logloss: 0.496061

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's binary_logloss: 0.642186
[4]	valid_0's binary_logloss: 0.602921
[6]	valid_0's binary_logloss: 0.569168
[8]	valid_0's binary_logloss: 0.544791
[10]	valid_0's binary_logloss: 0.526236
[12]	valid_0's binary_logloss: 0.509003
[14]	valid_0's binary_logloss: 0.490634
[16]	valid_0's binary_logloss: 0.480572
[18]	valid_0's binary_logloss: 0.481125
[20

...

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[6]	validation_0-logloss:0.59101
[8]	validation_0-logloss:0.58074
[10]	validation_0-logloss:0.57102
[12]	validation_0-logloss:0.56831
[14]	validation_0-logloss:0.56422
[16]	validation_0-logloss:0.55998
[18]	validation_0-logloss:0.55809
[20]	validation_0-logloss:0.55686
[22]	validation_0-logloss:0.55113
[24]	validation_0-logloss:0.54667
[26]	validation_0-logloss:0.54698
[28]	validation_0-logloss:0.54198
[30]	validation_0-logloss:0.54267
[32]	validation_0-logloss:0.53800
[34]	validation_0-logloss:0.53624
[36]	validation_0-logloss:0.53791
[38]	validation_0-logloss:0.53784
[40]	validation_0-logloss:0.53536
[42]	validation_0-logloss:0.53640
[44]	validation_0-logloss:0.53664
[46]	validation_0-logloss:0.53666
[48]	validation_0-logloss:0.53732

[0]	validation_0-logloss:0.65828
[2]	validation_0-logloss:0.60719
[4]	validation_0-logloss:0.57026


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[6]	validation_0-logloss:0.54228
[8]	validation_0-logloss:0.52190
[10]	validation_0-logloss:0.50554
[12]	validation_0-logloss:0.49450
[14]	validation_0-logloss:0.48509
[16]	validation_0-logloss:0.47864
[18]	validation_0-logloss:0.47198
[20]	validation_0-logloss:0.46779
[22]	validation_0-logloss:0.46245
[24]	validation_0-logloss:0.45974
[26]	validation_0-logloss:0.45714
[28]	validation_0-logloss:0.45344
[30]	validation_0-logloss:0.45090
[32]	validation_0-logloss:0.44906
[34]	validation_0-logloss:0.44641
[36]	validation_0-logloss:0.44421
[38]	validation_0-logloss:0.44036
[40]	validation_0-logloss:0.44032
[42]	validation_0-logloss:0.43999
[44]	validation_0-logloss:0.43701
[46]	validation_0-logloss:0.43555
[48]	validation_0-logloss:0.43428
[50]	validation_0-logloss:0.43116
[52]	validation_0-logloss:0.42940
[54]	validation_0-logloss:0.42798
[56]	validation_0-logloss:0.42710
[58]	validation_0-logloss:0.42682
[60]	validation_0-logloss:0.42596
[62]	validation_0-logloss:0.42586
[64]	validation_

0.768438595999205

In [56]:
data_test = data_test[data_test['id'].isin(test_id_list)]

In [57]:
demand_pred_lgbm = lgbm_clf.predict(data_test.drop(['id','demand','ln_demand'], axis=1))
demand_pred_catboost = catboost_clf.predict(data_test.drop(['id','demand','ln_demand'], axis=1))
demand_pred_xgb = xgb_clf.predict(data_test.drop(['id','demand','ln_demand'], axis=1))

In [58]:
#проверим первый вариант 
data_test['is_high'] = demand_pred_lgbm

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [59]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

In [60]:
from catboost import CatBoostRegressor

In [62]:
#первый вариант
model_high_lgb = lgb.LGBMRegressor(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40)    
model_low_lgb = lgb.LGBMRegressor(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40) 
#второй вариант
model_high_catboost = CatBoostRegressor(iterations=350,
                        learning_rate=0.1,
                        depth=10) 
model_low_catboost = CatBoostRegressor(iterations=350,
                        learning_rate=0.1,
                        depth=10)

#третий вариант
model_high_xgb = xgb.XGBRegressor(n_estimators=350,
                        learning_rate=0.1) 
model_low_xgb = xgb.XGBRegressor(n_estimators=350,
                        learning_rate=0.1)


In [63]:
y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)


y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [64]:
#обучим lgbm 
CV(X_train_high, y_train_high, 20, model_high_lgb, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_lgb, mean_absolute_error, kwargs)


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0935056
[4]	valid_0's l2: 0.0913959
[6]	valid_0's l2: 0.0909921
[8]	valid_0's l2: 0.0895232
[10]	valid_0's l2: 0.090102
[12]	valid_0's l2: 0.089432
[14]	valid_0's l2: 0.0881899
[16]	valid_0's l2: 0.0875513
[18]	valid_0's l2: 0.0872851
[20]	valid_0's l2: 0.0868224
[22]	valid_0's l2: 0.0875234
[24]	valid_0's l2: 0.0868326
[26]	valid_0's l2: 0.0872774
[28]	valid_0's l2: 0.0879864
[30]	valid_0's l2: 0.0876871

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0294831
[4]	valid_0's l2: 0.0297197
[6]	valid_0's l2: 0.0296546
[8]	valid_0's l2: 0.0297916
[10]	valid_0's l2: 0.0298655

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0197979
[4]	valid_0's l2: 0.0194948

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[2]	valid_0's l2: 0.0557583
[4]	valid_0's l2: 0.0554923
[6]	valid_0's l2: 0.0544295
[8]	valid_0's l2: 0.0545255
[10]	valid_0's l2: 0.0551621
[12]	valid_0's l2: 0.0549176
[14]	valid_0's l2: 0.0543552
[16]	valid_0's l2: 0.0551533
[18]	valid_0's l2: 0.0549813
[20]	valid_0's l2: 0.0547161
[22]	valid_0's l2: 0.0550885
[24]	valid_0's l2: 0.0544512

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0301032
[4]	valid_0's l2: 0.0298499
[6]	valid_0's l2: 0.0288624
[8]	valid_0's l2: 0.0280459
[10]	valid_0's l2: 0.0281453
[12]	valid_0's l2: 0.0278783
[14]	valid_0's l2: 0.027335
[16]	valid_0's l2: 0.0271978
[18]	valid_0's l2: 0.0272907
[20]	valid_0's l2: 0.0270232
[22]	valid_0's l2: 0.0268296
[24]	valid_0's l2: 0.0266084
[26]	valid_0's l2: 0.0265499
[28]	valid_0's l2: 0.0265756
[30]	valid_0's l2: 0.0266365
[32]	valid_0's l2: 0.026575
[34]	valid_0's l2: 0.0266097
[36]	valid_0's l2: 0.026615

[LightGBM] [War

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[2]	valid_0's l2: 0.0292084
[4]	valid_0's l2: 0.028897
[6]	valid_0's l2: 0.0286785
[8]	valid_0's l2: 0.0283703
[10]	valid_0's l2: 0.0285746
[12]	valid_0's l2: 0.0283903
[14]	valid_0's l2: 0.0281473
[16]	valid_0's l2: 0.0280162
[18]	valid_0's l2: 0.0279745
[20]	valid_0's l2: 0.0282257
[22]	valid_0's l2: 0.0282145
[24]	valid_0's l2: 0.0283063
[26]	valid_0's l2: 0.0283275

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.116013
[4]	valid_0's l2: 0.112813
[6]	valid_0's l2: 0.110843
[8]	valid_0's l2: 0.106851
[10]	valid_0's l2: 0.104165
[12]	valid_0's l2: 0.102513


...

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[28]	valid_0's l2: 0.00785661
[30]	valid_0's l2: 0.00779389
[32]	valid_0's l2: 0.00773128
[34]	valid_0's l2: 0.00768314
[36]	valid_0's l2: 0.00765682
[38]	valid_0's l2: 0.00767066
[40]	valid_0's l2: 0.00764164
[42]	valid_0's l2: 0.00764369
[44]	valid_0's l2: 0.007669
[46]	valid_0's l2: 0.0076637
[48]	valid_0's l2: 0.00761477
[50]	valid_0's l2: 0.00760697
[52]	valid_0's l2: 0.00760738
[54]	valid_0's l2: 0.00753848
[56]	valid_0's l2: 0.00752297
[58]	valid_0's l2: 0.00750405
[60]	valid_0's l2: 0.00748812
[62]	valid_0's l2: 0.00749451
[64]	valid_0's l2: 0.0075311
[66]	valid_0's l2: 0.0075246
[68]	valid_0's l2: 0.00749988
[70]	valid_0's l2: 0.00747273
[72]	valid_0's l2: 0.00743822
[74]	valid_0's l2: 0.00743557
[76]	valid_0's l2: 0.00745258
[78]	valid_0's l2: 0.00742869
[80]	valid_0's l2: 0.00744336
[82]	valid_0's l2: 0.00743432
[84]	valid_0's l2: 0.00742408
[86]	valid_0's l2: 0.00742065
[88]	valid_0's l2: 0.00742011
[90]	valid_0's l2: 0.00742055
[92]	valid_0's l2: 0.00742627
[94]	valid_0's 

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[2]	valid_0's l2: 0.0140718
[4]	valid_0's l2: 0.0125748
[6]	valid_0's l2: 0.0115518
[8]	valid_0's l2: 0.0104997
[10]	valid_0's l2: 0.00968716
[12]	valid_0's l2: 0.00913036
[14]	valid_0's l2: 0.0086268
[16]	valid_0's l2: 0.00836666
[18]	valid_0's l2: 0.00813015
[20]	valid_0's l2: 0.00788034
[22]	valid_0's l2: 0.00779529
[24]	valid_0's l2: 0.00760031
[26]	valid_0's l2: 0.00753778
[28]	valid_0's l2: 0.0074576
[30]	valid_0's l2: 0.00737777
[32]	valid_0's l2: 0.007308
[34]	valid_0's l2: 0.00724648
[36]	valid_0's l2: 0.00717645
[38]	valid_0's l2: 0.00712822
[40]	valid_0's l2: 0.00706679
[42]	valid_0's l2: 0.00703914
[44]	valid_0's l2: 0.0069954
[46]	valid_0's l2: 0.00692908
[48]	valid_0's l2: 0.00691367
[50]	valid_0's l2: 0.00691392
[52]	valid_0's l2: 0.00686912
[54]	valid_0's l2: 0.0068275
[56]	valid_0's l2: 0.00680933
[58]	valid_0's l2: 0.00678156
[60]	valid_0's l2: 0.00679134
[62]	valid_0's l2: 0.00677951
[64]	valid_0's l2: 0.00677054
[66]	valid_0's l2: 0.00674831
[68]	valid_0's l2: 0.006

0.05862374292657633

In [65]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

In [66]:
pred_high = model_high_lgb.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_lgb.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [67]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

In [68]:
lgb_pred = pd.DataFrame({'id': test_id_list})
lgb_pred = lgb_pred.merge(pred, on = 'id')
lgb_pred['demand'] = np.exp(lgb_pred['demand']) - 1
lgb_pred.head()

,id,demand
0,601,0.142878
1,697,0.154571
2,831,0.174244
3,875,1.068279
4,975,0.165231


In [69]:
lgb_pred.to_csv('lgb_pred.csv', index=False) 

In [70]:
#проверим второй вариант 
data_test['is_high'] = demand_pred_catboost

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [71]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

In [72]:
y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)


y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [73]:
#обучим catboost 
CV(X_train_high, y_train_high, 20, model_high_catboost, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_catboost, mean_absolute_error, kwargs)


0:	learn: 0.1499180	test: 0.3111270	best: 0.3111270 (0)	total: 10.7ms	remaining: 3.74s
2:	learn: 0.1440408	test: 0.3091465	best: 0.3091465 (2)	total: 21.8ms	remaining: 2.52s
4:	learn: 0.1397760	test: 0.3082666	best: 0.3082666 (4)	total: 33.9ms	remaining: 2.34s
6:	learn: 0.1351371	test: 0.3070231	best: 0.3070231 (6)	total: 44.8ms	remaining: 2.2s
8:	learn: 0.1326333	test: 0.3063169	best: 0.3063169 (8)	total: 50.2ms	remaining: 1.9s
10:	learn: 0.1286862	test: 0.3053686	best: 0.3053686 (10)	total: 61.3ms	remaining: 1.89s
12:	learn: 0.1258944	test: 0.3046316	best: 0.3046155 (11)	total: 70.5ms	remaining: 1.83s
14:	learn: 0.1223940	test: 0.3044863	best: 0.3044863 (14)	total: 81.5ms	remaining: 1.82s
16:	learn: 0.1194809	test: 0.3034437	best: 0.3034437 (16)	total: 96.4ms	remaining: 1.89s
18:	learn: 0.1168457	test: 0.3019628	best: 0.3019628 (18)	total: 106ms	remaining: 1.85s
20:	learn: 0.1146809	test: 0.3014481	best: 0.3014481 (20)	total: 113ms	remaining: 1.77s
22:	learn: 0.1129409	test: 0.30123

0.059993276379430346

In [74]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

pred_high = model_high_catboost.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_catboost.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [75]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

catboost_pred = pd.DataFrame({'id': test_id_list})
catboost_pred = catboost_pred.merge(pred, on = 'id')
catboost_pred['demand'] = np.exp(catboost_pred['demand']) - 1
catboost_pred.head()

,id,demand
0,601,0.190386
1,697,0.134276
2,831,0.194088
3,875,1.157346
4,975,0.128305


In [76]:
catboost_pred.to_csv('catboost_pred.csv', index=False) 
#стало немного лучше на тесте 

In [77]:
#проверим третий вариант 
data_test['is_high'] = demand_pred_xgb

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [78]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

In [79]:
y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)


y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [81]:
 #обучим xgb
CV(X_train_high, y_train_high, 20, model_high_xgb, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_xgb, mean_absolute_error, kwargs)


[0]	validation_0-rmse:0.41000
[2]	validation_0-rmse:0.38052
[4]	validation_0-rmse:0.35860
[6]	validation_0-rmse:0.34495
[8]	validation_0-rmse:0.33443
[10]	validation_0-rmse:0.32713
[12]	validation_0-rmse:0.31975
[14]	validation_0-rmse:0.31452
[16]	validation_0-rmse:0.31182
[18]	validation_0-rmse:0.30949
[20]	validation_0-rmse:0.30705
[22]	validation_0-rmse:0.30517
[24]	validation_0-rmse:0.30400
[26]	validation_0-rmse:0.30345
[28]	validation_0-rmse:0.30204


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[30]	validation_0-rmse:0.30107
[32]	validation_0-rmse:0.30082
[34]	validation_0-rmse:0.30060
[36]	validation_0-rmse:0.30082
[38]	validation_0-rmse:0.30103
[40]	validation_0-rmse:0.30116
[42]	validation_0-rmse:0.30153
[43]	validation_0-rmse:0.30143

[0]	validation_0-rmse:0.28036
[2]	validation_0-rmse:0.24795
[4]	validation_0-rmse:0.22591
[6]	validation_0-rmse:0.21161
[8]	validation_0-rmse:0.20084
[10]	validation_0-rmse:0.18900
[12]	validation_0-rmse:0.18359
[14]	validation_0-rmse:0.18041
[16]	validation_0-rmse:0.17747
[18]	validation_0-rmse:0.17607
[20]	validation_0-rmse:0.17258
[22]	validation_0-rmse:0.17029
[24]	validation_0-rmse:0.16982


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[26]	validation_0-rmse:0.16978
[28]	validation_0-rmse:0.16963
[30]	validation_0-rmse:0.16970
[32]	validation_0-rmse:0.16970
[34]	validation_0-rmse:0.16944
[36]	validation_0-rmse:0.17022
[38]	validation_0-rmse:0.17085
[40]	validation_0-rmse:0.17145
[42]	validation_0-rmse:0.17154
[43]	validation_0-rmse:0.17152

[0]	validation_0-rmse:0.25382
[2]	validation_0-rmse:0.22347
[4]	validation_0-rmse:0.20113
[6]	validation_0-rmse:0.18479
[8]	validation_0-rmse:0.17331
[10]	validation_0-rmse:0.16206
[12]	validation_0-rmse:0.15588
[14]	validation_0-rmse:0.15166


...

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[12]	validation_0-rmse:0.13426
[14]	validation_0-rmse:0.12103
[16]	validation_0-rmse:0.11124
[18]	validation_0-rmse:0.10479
[20]	validation_0-rmse:0.10015
[22]	validation_0-rmse:0.09667
[24]	validation_0-rmse:0.09434
[26]	validation_0-rmse:0.09251
[28]	validation_0-rmse:0.09120
[30]	validation_0-rmse:0.09013
[32]	validation_0-rmse:0.08955
[34]	validation_0-rmse:0.08933
[36]	validation_0-rmse:0.08876
[38]	validation_0-rmse:0.08857
[40]	validation_0-rmse:0.08814
[42]	validation_0-rmse:0.08762
[44]	validation_0-rmse:0.08746
[46]	validation_0-rmse:0.08726
[48]	validation_0-rmse:0.08721
[50]	validation_0-rmse:0.08718
[52]	validation_0-rmse:0.08694
[54]	validation_0-rmse:0.08692
[56]	validation_0-rmse:0.08693
[58]	validation_0-rmse:0.08696
[60]	validation_0-rmse:0.08694
[62]	validation_0-rmse:0.08674
[64]	validation_0-rmse:0.08679
[66]	validation_0-rmse:0.08673
[68]	validation_0-rmse:0.08670
[70]	validation_0-rmse:0.08658
[72]	validation_0-rmse:0.08648
[74]	validation_0-rmse:0.08641
[76]	val

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[12]	validation_0-rmse:0.13317
[14]	validation_0-rmse:0.11813
[16]	validation_0-rmse:0.10691
[18]	validation_0-rmse:0.09956
[20]	validation_0-rmse:0.09400
[22]	validation_0-rmse:0.09082
[24]	validation_0-rmse:0.08836
[26]	validation_0-rmse:0.08631
[28]	validation_0-rmse:0.08522
[30]	validation_0-rmse:0.08446
[32]	validation_0-rmse:0.08394
[34]	validation_0-rmse:0.08374
[36]	validation_0-rmse:0.08348
[38]	validation_0-rmse:0.08330
[40]	validation_0-rmse:0.08309
[42]	validation_0-rmse:0.08300
[44]	validation_0-rmse:0.08292
[46]	validation_0-rmse:0.08282
[48]	validation_0-rmse:0.08276
[50]	validation_0-rmse:0.08257
[52]	validation_0-rmse:0.08258
[54]	validation_0-rmse:0.08226
[56]	validation_0-rmse:0.08222
[58]	validation_0-rmse:0.08211
[60]	validation_0-rmse:0.08204
[62]	validation_0-rmse:0.08192
[64]	validation_0-rmse:0.08200
[66]	validation_0-rmse:0.08204
[68]	validation_0-rmse:0.08211
[70]	validation_0-rmse:0.08202
[72]	validation_0-rmse:0.08220


0.06287503715363689

In [82]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

pred_high = model_high_xgb.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_xgb.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [83]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

xgb_pred = pd.DataFrame({'id': test_id_list})
xgb_pred = xgb_pred.merge(pred, on = 'id')
xgb_pred['demand'] = np.exp(xgb_pred['demand']) - 1
xgb_pred.head()

,id,demand
0,601,0.164855
1,697,1.042937
2,831,0.177677
3,875,1.202018
4,975,1.032284


In [84]:
xgb_pred.to_csv('xgb_pred.csv', index=False) 
#работает хуже всего

In [86]:
#возьмем катбуст как классификатор и lgbm как регрессор 
data_test['is_high'] = demand_pred_catboost

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)

y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [88]:
model_high_lgb = lgb.LGBMRegressor(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40)    
model_low_lgb = lgb.LGBMRegressor(
                        n_estimators=350,
                        learning_rate=0.1,
                        feature_fraction=0.7,
                        subsample=0.4,
                        num_leaves=40) 

model_high_catboost = CatBoostRegressor(iterations=350,
                        learning_rate=0.1,
                        depth=10) 
model_low_catboost = CatBoostRegressor(iterations=350,
                        learning_rate=0.1,
                        depth=10)

In [89]:
#обучим lgbm
CV(X_train_high, y_train_high, 20, model_high_lgb, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_lgb, mean_absolute_error, kwargs)


[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0935056
[4]	valid_0's l2: 0.0913959
[6]	valid_0's l2: 0.0909921
[8]	valid_0's l2: 0.0895232
[10]	valid_0's l2: 0.090102
[12]	valid_0's l2: 0.089432
[14]	valid_0's l2: 0.0881899
[16]	valid_0's l2: 0.0875513
[18]	valid_0's l2: 0.0872851
[20]	valid_0's l2: 0.0868224
[22]	valid_0's l2: 0.0875234
[24]	valid_0's l2: 0.0868326
[26]	valid_0's l2: 0.0872774
[28]	valid_0's l2: 0.0879864
[30]	valid_0's l2: 0.0876871

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0294831
[4]	valid_0's l2: 0.0297197
[6]	valid_0's l2: 0.0296546
[8]	valid_0's l2: 0.0297916
[10]	valid_0's l2: 0.0298655

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0197979
[4]	valid_0's l2: 0.0194948

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[4]	valid_0's l2: 0.0554923
[6]	valid_0's l2: 0.0544295
[8]	valid_0's l2: 0.0545255
[10]	valid_0's l2: 0.0551621
[12]	valid_0's l2: 0.0549176
[14]	valid_0's l2: 0.0543552
[16]	valid_0's l2: 0.0551533
[18]	valid_0's l2: 0.0549813
[20]	valid_0's l2: 0.0547161
[22]	valid_0's l2: 0.0550885
[24]	valid_0's l2: 0.0544512

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[2]	valid_0's l2: 0.0301032
[4]	valid_0's l2: 0.0298499
[6]	valid_0's l2: 0.0288624
[8]	valid_0's l2: 0.0280459
[10]	valid_0's l2: 0.0281453
[12]	valid_0's l2: 0.0278783
[14]	valid_0's l2: 0.027335
[16]	valid_0's l2: 0.0271978
[18]	valid_0's l2: 0.0272907
[20]	valid_0's l2: 0.0270232
[22]	valid_0's l2: 0.0268296
[24]	valid_0's l2: 0.0266084
[26]	valid_0's l2: 0.0265499
[28]	valid_0's l2: 0.0265756
[30]	valid_0's l2: 0.0266365
[32]	valid_0's l2: 0.026575
[34]	valid_0's l2: 0.0266097
[36]	valid_0's l2: 0.026615

[LightGBM] [Warning] feature_fraction is se

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[6]	valid_0's l2: 0.0286785
[8]	valid_0's l2: 0.0283703
[10]	valid_0's l2: 0.0285746
[12]	valid_0's l2: 0.0283903
[14]	valid_0's l2: 0.0281473
[16]	valid_0's l2: 0.0280162
[18]	valid_0's l2: 0.0279745
[20]	valid_0's l2: 0.0282257
[22]	valid_0's l2: 0.0282145
[24]	valid_0's l2: 0.0283063
[26]	valid_0's l2: 0.0283275

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7


...

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

[18]	valid_0's l2: 0.0126338
[20]	valid_0's l2: 0.0124224
[22]	valid_0's l2: 0.0124071
[24]	valid_0's l2: 0.0122566
[26]	valid_0's l2: 0.0121716
[28]	valid_0's l2: 0.0122299
[30]	valid_0's l2: 0.0122141
[32]	valid_0's l2: 0.012169
[34]	valid_0's l2: 0.012135
[36]	valid_0's l2: 0.0121275
[38]	valid_0's l2: 0.0121023
[40]	valid_0's l2: 0.012036
[42]	valid_0's l2: 0.0120073
[44]	valid_0's l2: 0.0119966
[46]	valid_0's l2: 0.0119585
[48]	valid_0's l2: 0.0119264
[50]	valid_0's l2: 0.0118634
[52]	valid_0's l2: 0.011846
[54]	valid_0's l2: 0.0118095
[56]	valid_0's l2: 0.0118094
[58]	valid_0's l2: 0.0117639
[60]	valid_0's l2: 0.0117419
[62]	valid_0's l2: 0.0117444
[64]	valid_0's l2: 0.0117425
[66]	valid_0's l2: 0.0117492
[68]	valid_0's l2: 0.01172
[70]	valid_0's l2: 0.0117124
[72]	valid_0's l2: 0.0116926
[74]	valid_0's l2: 0.0116941
[76]	valid_0's l2: 0.0116992
[78]	valid_0's l2: 0.0117126
[80]	valid_0's l2: 0.0117209

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will b

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[12]	valid_0's l2: 0.00913036
[14]	valid_0's l2: 0.0086268
[16]	valid_0's l2: 0.00836666
[18]	valid_0's l2: 0.00813015
[20]	valid_0's l2: 0.00788034
[22]	valid_0's l2: 0.00779529
[24]	valid_0's l2: 0.00760031
[26]	valid_0's l2: 0.00753778
[28]	valid_0's l2: 0.0074576
[30]	valid_0's l2: 0.00737777
[32]	valid_0's l2: 0.007308
[34]	valid_0's l2: 0.00724648
[36]	valid_0's l2: 0.00717645
[38]	valid_0's l2: 0.00712822
[40]	valid_0's l2: 0.00706679
[42]	valid_0's l2: 0.00703914
[44]	valid_0's l2: 0.0069954
[46]	valid_0's l2: 0.00692908
[48]	valid_0's l2: 0.00691367
[50]	valid_0's l2: 0.00691392
[52]	valid_0's l2: 0.00686912
[54]	valid_0's l2: 0.0068275
[56]	valid_0's l2: 0.00680933
[58]	valid_0's l2: 0.00678156
[60]	valid_0's l2: 0.00679134
[62]	valid_0's l2: 0.00677951
[64]	valid_0's l2: 0.00677054
[66]	valid_0's l2: 0.00674831
[68]	valid_0's l2: 0.00674146
[70]	valid_0's l2: 0.00672912
[72]	valid_0's l2: 0.00670521
[74]	valid_0's l2: 0.00668492
[76]	valid_0's l2: 0.00667708
[78]	valid_0's l

0.05862374292657633

In [90]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

pred_high = model_high_lgb.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_lgb.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [91]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

pred1 = pd.DataFrame({'id': test_id_list})
pred1 = pred1.merge(pred, on = 'id')
pred1['demand'] = np.exp(pred1['demand']) - 1
pred1.head()

,id,demand
0,601,0.142878
1,697,0.154571
2,831,0.174244
3,875,1.068279
4,975,0.165231


In [92]:
pred1.to_csv('lgbm_cat_pred.csv', index=False) 
#работает лучше, чем все предыдущие варианты 

In [94]:
#возьмем катбуст как регрессор и lgbm как классификатор
data_test['is_high'] = demand_pred_lgbm

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [95]:
high_ids = list(data_test['id'].loc[data_test['is_high'] == 1])
low_ids = list(data_test['id'].loc[data_test['is_high'] == 0])

y_train_high = data_high.ln_demand
X_train_high = data_high.drop(['id','ln_demand','is_high', 'demand'], axis=1)

y_train_low = data_low.ln_demand
X_train_low = data_low.drop(['id','ln_demand','is_high', 'demand'], axis=1)

In [96]:
#обучим catboost
CV(X_train_high, y_train_high, 20, model_high_catboost, mean_absolute_error, kwargs)
CV(X_train_low, y_train_low, 20, model_low_catboost, mean_absolute_error, kwargs)


0:	learn: 0.1499180	test: 0.3111270	best: 0.3111270 (0)	total: 14ms	remaining: 4.89s
2:	learn: 0.1440408	test: 0.3091465	best: 0.3091465 (2)	total: 25.7ms	remaining: 2.97s
4:	learn: 0.1397760	test: 0.3082666	best: 0.3082666 (4)	total: 36.1ms	remaining: 2.49s
6:	learn: 0.1351371	test: 0.3070231	best: 0.3070231 (6)	total: 56.4ms	remaining: 2.76s
8:	learn: 0.1326333	test: 0.3063169	best: 0.3063169 (8)	total: 62.6ms	remaining: 2.37s
10:	learn: 0.1286862	test: 0.3053686	best: 0.3053686 (10)	total: 72.4ms	remaining: 2.23s
12:	learn: 0.1258944	test: 0.3046316	best: 0.3046155 (11)	total: 82.6ms	remaining: 2.14s
14:	learn: 0.1223940	test: 0.3044863	best: 0.3044863 (14)	total: 92.7ms	remaining: 2.07s
16:	learn: 0.1194809	test: 0.3034437	best: 0.3034437 (16)	total: 102ms	remaining: 2s
18:	learn: 0.1168457	test: 0.3019628	best: 0.3019628 (18)	total: 112ms	remaining: 1.95s
20:	learn: 0.1146809	test: 0.3014481	best: 0.3014481 (20)	total: 119ms	remaining: 1.86s
22:	learn: 0.1129409	test: 0.3012300	b

0.059993276379430346

In [97]:
test_high = data_test.loc[data_test['is_high'] == 1]
test_low = data_test.loc[data_test['is_high'] == 0]

pred_high = model_high_catboost.predict(test_high.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))
pred_low = model_low_catboost.predict(test_low.drop(['id','demand', 'ln_demand', 'is_high'], axis=1))

In [98]:
high = pd.DataFrame({'id': high_ids, 'demand': pred_high})
low = pd.DataFrame({'id': low_ids, 'demand': pred_low})
pred = pd.concat([high, low])

pred2 = pd.DataFrame({'id': test_id_list})
pred2 = pred2.merge(pred, on = 'id')
pred2['demand'] = np.exp(pred2['demand']) - 1
pred2.head()

,id,demand
0,601,0.190386
1,697,0.134276
2,831,0.194088
3,875,1.157346
4,975,0.128305


In [99]:
pred2.to_csv('lgbm_cat_pred2.csv', index=False) 
#работает хуже

#лучший результат: lgbm регрессор + catboost классификатор 